In [99]:
# Builds the minimum input control configuration (CF) from a given maximum matching
# Inputs are:
#    G, the network digraph
#    MM, another digraph consisting of the same nodes as G but only the edges contained in the maximum matching

# The procedure for this function is depicted schematically in Wang (2012) Fig. 2

import networkx as nx
import numpy as np


def BuildCF(G,MM):
    matching = MM.edges()
    zz = list(zip(*matching)) #split tuple list into "out" list and "in" list (just MM links)
    MMoutNodes = np.array(zz[0])
    MMinNodes = np.array(zz[1])
    
    allEdges = G.edges()
    zz = list(zip(*allEdges)) #split tuple list into "out" list and "in" list (all links)
    GoutNodes = np.array(zz[0])
    GinNodes = np.array(zz[1])
    
    # Find MDNS:
    
    MDNS = MM.nodes() #will be pruned down to Minimum Driver Node Set
    for lnk in matching: #if downstream end of edge is in MDNS, pop it
        try: 
            MDNS.remove(lnk[1])
        except ValueError:
            pass
    print('Driver nodes: ', MDNS)
    
    # Find Stems:
    
    nStem = len(MDNS)
    
    
    stems = {} #dict of stems: key is index of MDNS, value is list of nodes
    
    for s in range(0,nStem):
        stems.setdefault(s, []) #initializes to empty list if value for s is undefined, does nothing if it's already defined
        stems[s].append(MDNS[s])
        searchval = MDNS[s]
        while 1:
            searchval = stems[s][-1] #set to last item in the stem
            ii = np.where(MMoutNodes == searchval)[0] #returns indices of all MM edges which start from node searchval
            if len(ii) > 1:
                print('Stem appears to have a fork in it!')
                break
            elif len(ii) == 0: #in this case we've reached the end of the stem
                break
            else:
                if MMinNodes[ii[0]] in stems[s]:
                    print("Stem has looped back on itself: If you're reading this you need to code up a way to deal with it")
                    break
                else:
                    stems[s].append(MMinNodes[ii[0]]) #add next node of the stem to dict value
        
    # Find Cycles:
        
    listCycles = list(nx.simple_cycles(MM))
    nCycle = len(listCycles)
    cycles = {} #dict of cycles
    for c in range(0,nCycle):
        cycles[c] = listCycles[c]
    
    # Find Additional Links (AL)
    
    AL = {} #AL dict, keys are stem indices and values are edge tuples
    linkedCycles = {} #stores the indices of cycles linked to each stem
    for s in range(0,nStem):
        AL.setdefault(s, [])
        linkedCycles.setdefault(s, [])
        unlinkedCycles = list(cycles.keys())
        thisStem = stems[s]
        for n in thisStem:
            ii = np.where(GoutNodes == n)[0] #returns indices of all edges which start from node n
            if len(ii) == 0:
                break #end of the stem
            for l in ii: #loop through edges coming out of n
                outN = GoutNodes[l]
                inN = GinNodes[l]
                if (inN in thisStem) == 0: #skip the edge that's in the stem
                    for c in unlinkedCycles:
                        if inN in cycles[c]: #check if link goes to an as-yet unlinked cycle
                            thisEdge = tuple([outN,inN])
                            AL[s].append(thisEdge)
                            unlinkedCycles.remove(c)
                            linkedCycles[s].append(c)
            if len(unlinkedCycles) == 0:
                break
                
    print("Stems: ", stems)
    print("Cycles: ", cycles)
    print("Additional Links: ", AL)
    print("Linked Cycles: ", linkedCycles)
        
    

In [100]:
# Test the function on the sample graph from Wang's paper (using the specific maximum matching depicted)
import TestNetworks as TN
G = TN.testNet(4)
MM = nx.DiGraph()
MM.add_nodes_from(range(1,17))
MM.add_edges_from([(1,16),(16,9),(9,12),(5,6),(6,7),(7,8),(8,5),(2,3),(13,14),(14,15),(15,13),(10,11),(11,10)])

BuildCF(G,MM)




Driver nodes:  [1, 2, 4]
Stems:  {0: [1, 16, 9, 12], 1: [2, 3], 2: [4]}
Cycles:  {0: [13, 14, 15], 1: [10, 11], 2: [8, 5, 6, 7]}
Additional Links:  {0: [(16, 5), (16, 13), (9, 10)], 1: [(2, 13)], 2: []}
Linked Cycles:  {0: [2, 0, 1], 1: [0], 2: []}
